    Universidad Nacional Autónoma de Honduras (UNAH)
    I PAC 2021   
    Examen de reposición de la tercera unidad
    Clase: Bases de Datos I
    Sección: 0701
    Catedrático: José Manuel Inestroza Murillo
    Alumno:  Kenneth Leonel Cruz Ordoñez           -   20141010391

In [1]:
from MySQLEngine import MySQLEngine
from ConfigConnection import Configuration
import re

In [2]:
config = Configuration(
            host="localhost", 
            port="3306", 
            user="admin", 
            password="admin"
        )
engine = MySQLEngine(config)

DB_NAME = 'SudokuDB'

VIEWS = {}
TABLES = {}
TRIGERS = {}
FUNCTIONS = {}
PROCEDURES = {}

add_user = {}
add_sudokuboard = {}

La conexión ha sido: Satisfactoria


## Tablas de la base de datos

Se describen las sentencias de definición de la base de datos.

In [ ]:
TABLES['User'] = (
    "CREATE TABLE User( "
    "    id SERIAL PRIMARY KEY, "
    "    tex_nickname VARCHAR(30) NOT NULL UNIQUE COMMENT 'Nombre de ingreso para el usuario',"
    "    CHECK(tex_nickname RLIKE '[a-zA-Z\d]{4,30}' ), "
    "    bit_rol BIT(1) DEFAULT 0 NOT NULL COMMENT '0 Usuario | 1 Administrador; Tipos de rol para acceso al sistema', " 
    "    tex_password TEXT NOT NULL COMMENT 'Contraseña', "
    "    bit_state BIT(1) DEFAULT 1 NOT NULL COMMENT '0 deshabilitado| 1 habilitado. Dar de baja a los usuarios, de esta forma se evita eliminar a los usuarios de forma explícita de la base de datos' "
    ")COMMENT 'Información general sobre el usuario'; "
)

TABLES['Login'] = (
    " CREATE TABLE Login( "
    "     id SERIAL PRIMARY KEY, "
    "     id_user_fk BIGINT UNSIGNED NOT NULL COMMENT 'Referencia hacia user', "
    "     tim_date TIMESTAMP DEFAULT NOW() COMMENT 'Hora de ingreso al sistema', "
    "     FOREIGN KEY (id_user_fk) REFERENCES User(id) "
    " )COMMENT 'Sistema de acceso al sistema'; "
)

TABLES['SudokuBoard'] = (
       " CREATE TABLE SudokuBoard( "
    "     id SERIAL PRIMARY KEY, "
    "     tex_board VARCHAR(89) NOT NULL UNIQUE COMMENT 'Contiene la información inicial de un tablero, para luego ser rellenado en el Board de la aplicación de escritorio' "
    " ) COMMENT 'Contiene la información de los tableros que pueden ser cargados en la aplicación'; "
)

TABLES['Game'] = (
    " CREATE TABLE Game( "
    "     id SERIAL PRIMARY KEY, "
    "     id_user_fk BIGINT UNSIGNED NOT NULL COMMENT 'Referencia', "
    "     id_sudokuboard_fk BIGINT UNSIGNED NOT NULL COMMENT 'Referencia hacia los archivos que contiene los tableros', "
    "     blo_file BLOB NOT NULL COMMENT 'Archivo .sudoku que genera el programa cada vez que inicia un tablero', "
    "     tim_time TIME NOT NULL COMMENT 'hh:mm:ss Minutos transcurridos tras iniciar una partidar o tras continuar una partida pausada', "
    "     tim_date TIMESTAMP DEFAULT NOW() NOT NULL COMMENT 'último estado en el que es almacenado el tablero en la base de datos', "
    "  "
    "     FOREIGN KEY (id_user_fk) REFERENCES User(id), "
    "     FOREIGN KEY (id_sudokuboard_fk) REFERENCES SudokuBoard(id) "
    " )COMMENT 'Tablero del juego'; ")

TABLES['LogOff'] = (
    " CREATE TABLE LogOff( "
    "     id SERIAL PRIMARY KEY, "
    "     id_user_fk BIGINT UNSIGNED NOT NULL COMMENT 'Referencia hacia usuario', "
    "     tim_date TIMESTAMP DEFAULT NOW() NOT NULL COMMENT 'Tiempo en el que cierre de sesión un usuario', "
    "  "
    "     FOREIGN KEY (id_user_fk) REFERENCES User(id) "
    " )COMMENT 'Cierre de sesión por parte de un usuario'; "
)


TABLES['State'] = (
    " CREATE TABLE State( "
    "     id SERIAL PRIMARY KEY, "
    "     id_game_fk BIGINT UNSIGNED NOT NULL COMMENT 'Referencia hacia la entidad Juego', "
    "     cod_state ENUM('nuevo', 'pausado', 'finalizado', 'derrota', 'continuar') DEFAULT 'nuevo' NOT NULL  "
    "     COMMENT 'Estados del juego, el estado finalizado significa haber concluido el juego con éxito, derrota es haber abandonado la partida', "
    "     tim_date TIMESTAMP DEFAULT NOW() NOT NULL COMMENT 'último estado en el que es almacenado el tablero en la base de datos', "
    "  "
    "     FOREIGN KEY (id_game_fk) REFERENCES Game(id) "
    " )COMMENT 'Estados en los que puede estar el tablero de juego para un jugador'; "
)

TABLES['Binacle'] = (
    " CREATE TABLE Binacle( "
    "     id SERIAL PRIMARY KEY,  "
    "     tex_nickname TINYTEXT NOT NULL COMMENT 'Descripción del nombre del usuario',  "
    "     tex_description TEXT NOT NULL COMMENT 'Descripción de la acción que realiza un usuario en el sistema', "
    "     tim_date TIMESTAMP NOT NULL DEFAULT NOW() COMMENT 'Tiempo exacto en el que se realizó la acción' "
    "      "
    " )COMMENT = 'Almacena acciones realizadas por los usuarios'; "
)

## Vistas para la base de datos

Definición de las vistas

In [ ]:
VIEWS['vw_GetLastLoginUser'] = (
    " CREATE VIEW vw_GetLastLoginUser "
    "     AS  "
    "     SELECT  "
    "         login.id AS id, "
    "         User.tex_nickname AS name, "
    "         User.bit_rol AS rol  "
    "     FROM  "
    "         User "
    "     INNER JOIN  "
    "         ( "
    "             SELECT  "
    "                 id_user_fk AS id "
    "             FROM  "
    "                 Login  "
    "             ORDER BY  "
    "                 tim_date DESC   "
    "             LIMIT 1 "
    "         ) AS login ON User.id = login.id "
    " ; "
)

### Funciones para la base de datos

Definición de las funciones


In [ ]:
FUNCTIONS['fn_compareData'] = (
    "     CREATE FUNCTION fn_compareData(pyNickname TEXT, pyPassword TEXT) RETURNS TEXT "
    "     BEGIN "
    "  "
    "     SET @nicknameResult = IF(pyNickname IN (SELECT tex_nickname FROM User), 1, 0); "
    "     SET @password = ( "
    "         SELECT "
    "             tex_password "
    "         FROM "
    "             User "
    "         WHERE "
    "             tex_nickname = pyNickname "
    "     ); "
    "     SET @passwordResult = IF(@password = HEX(AES_ENCRYPT(pyPassword, pyNickname)), 1, 0); "
    "     SET @rolResult = IF((SELECT bit_rol FROM User WHERE tex_nickname = pyNickname) = 1, 1, 0); "
    "     SET @newPasswordResult = IF((SELECT HEX(AES_ENCRYPT(pyNickname, pyNickname)) FROM User WHERE tex_nickname = pyNickname) = @password, 1, 0); "
    "     SET @userState = IF((SELECT bit_state FROM User WHERE tex_nickname = pyNickname) = 1, 1, 0); "
    "     SET @result = (SELECT CONCAT(@nicknameResult, ' ', @passwordResult, ' ' ,@rolResult, ' ', @newPasswordResult, ' ', @userState)); "
    "     RETURN @result; "
    "  "
    "     END "
)

FUNCTIONS['fn_getNickNameByState'] = (
     "    CREATE FUNCTION fn_getNickNameByState(id_game_fk BIGINT UNSIGNED) RETURNS VARCHAR(40) "
     "    BEGIN  "
     "        RETURN ( "
     "                    SELECT   "
     "                        User.tex_nickname AS nickname "
     "                    FROM "
     "                        User "
     "                    INNER JOIN  "
     "                        Game ON User.id = Game.id_user_fk "
     "                    WHERE  "
     "                        Game.id = id_game_fk "
     "                ) "
     "            ; "
     "    END "
)

FUNCTIONS['fn_getNicknameById'] = (
    "     CREATE FUNCTION fn_getNicknameById(id_user_fk BIGINT UNSIGNED) RETURNS VARCHAR(40) "
    "     BEGIN  "
    "         RETURN ( "
    "                     SELECT   "
    "                         tex_nickname AS nickname "
    "                     FROM "
    "                         User "
    "                     WHERE  "
    "                         id = id_user_fk "
    "                 ) "
    "             ; "
    "     END "
)

### Procedimientos almacenados para la base de datos

Definición de los procedimientos almacenados


In [ ]:
PROCEDURES['sp_updatePassword'] = (
    " CREATE PROCEDURE sp_updatePassword(IN pyOldNickname TEXT, IN pyNewNickname TEXT) "
    " BEGIN "
    "      "
    "     SET @oldPassword = ( "
    "         SELECT  "
    "             AES_DECRYPT(UNHEX(tex_password), pyOldNickname)  "
    "         FROM  "
    "             User  "
    "         WHERE  "
    "             tex_nickname = pyOldNickname "
    "         ); "
    "  "
    "     UPDATE  "
    "         User  "
    "     SET  "
    "         tex_password = HEX(AES_ENCRYPT(@oldPassword, pyNewNickname))  "
    "     WHERE  "
    "         tex_nickname = pyOldNickname; "
    "  "
    " END "
)

## Triggers para la base de datos

Definición de los Triggers

In [ ]:
TRIGERS['tg_createBoard'] = (
    "     CREATE TRIGGER tg_createBoard  "
    "         AFTER INSERT "
    "         ON State FOR EACH ROW "
    "     BEGIN   "
    "         INSERT INTO Binacle(tex_nickname, tex_description) VALUES "
    "             ( "
    "                 fn_getNickNameByState(new.id_game_fk) "
    "                 ,  "
    "  "
    "                 ( "
    "                     SELECT  "
    "                         CASE "
    "                             WHEN  new.cod_state = 1 THEN  'Creó un nuevo tablero' "
    "                             WHEN  new.cod_state = 2 THEN 'Pausó el juego' "
    "                             WHEN  new.cod_state = 3 THEN 'Finalizó con éxito la partida' "
    "                             WHEN  new.cod_state = 4 THEN 'Perdió la partida' "
    "                             WHEN  new.cod_state = 5 THEN 'Continuó la partida en pausa' "
    "                         END  "
    "                 ) "
    "             ) "
    "         ; "
    "     END "
)

TRIGERS['tg_login'] = (
    "     CREATE TRIGGER tg_login "
    "         AFTER INSERT  "
    "         ON Login FOR EACH ROW "
    "     BEGIN  "
    "         INSERT INTO Binacle(tex_nickname, tex_description) VALUES "
    "             ( "
    "                 fn_getNicknameById(new.id_user_fk),  "
    "                 'El usuario inició sesión' "
    "             ) "
    "         ; "
    "     END "
)

TRIGERS['tg_logOff'] = (
    "     CREATE TRIGGER tg_logOff "
    "         AFTER INSERT  "
    "         ON LogOff FOR EACH ROW "
    "     BEGIN  "
    "         INSERT INTO Binacle(tex_nickname, tex_description) VALUES "
    "             ( "
    "                 fn_getNicknameById(new.id_user_fk),  "
    "                 'El usuario cerró sesión' "
    "             ) "
    "         ; "
    "     END  "
)

TRIGERS['tg_createUser'] = (
    "     CREATE TRIGGER tg_createUser "
    "         AFTER INSERT  "
    "         ON User FOR EACH ROW "
    "     BEGIN  "
    "         INSERT INTO Binacle(tex_nickname, tex_description) VALUES "
    "             ( "
    "                 'admin',  "
    "                 CONCAT('El usuario ', new.tex_nickname, ' ha sido creado') "
    "                  "
    "             ) "
    "         ; "
    "     END "
)

## Instancias para la creación de la base de dato, tablas, funciones, triggers

Se crean instancias para hacer llamado a los diccionarios que contienen la definición de las operaciones

In [ ]:
engine.createDatabase(DB_NAME)

In [3]:
engine.useDB(DB_NAME)

Query OK USE: SudokuDB


In [ ]:
engine.createSentence(TABLES)

In [ ]:
engine.createSentence(VIEWS)

In [ ]:
engine.createSentence(FUNCTIONS)

In [ ]:
engine.createSentence(PROCEDURES)

In [ ]:
engine.createSentence(TRIGERS)

## Datos de inserción para la base de datos


Se describen los datos de ejemplo a insertar a la base de datos

In [ ]:
add_user['1'] = (
    " INSERT INTO User(tex_nickname, tex_password, bit_rol) VALUES "
    "     ('admin', HEX(AES_ENCRYPT('admin', 'admin')), 1) "
    " ; "
)

add_user['2'] = (
    "INSERT INTO User(tex_nickname, tex_password) VALUES"
    "('Renata', 'admin'),"
    "('Angie', 'admin'),"
    "('Clarissa', 'admin'),"
    "('Melissa', 'admin')"
    ";"
)

add_sudokuboard['1'] = (
     "INSERT INTO SudokuBoard(tex_board) VALUES  "
     "    ('000846032\n630297100\n284015960\n400508300\n398004005\n001009080\n160000070\n840601053\n700000004'),  "
     "    ('210000400\n380400702\n000720000\n024806900\n000000000\n001203540\n000058000\n903004028\n008000057'),  "
     "    ('008100000\n602003094\n709080006\n800320060\n061070350\n030014009\n900040208\n120800905\n000001600'),  "
     "    ('010007200\n009024003\n007900060\n140070006\n900208001\n500040038\n090001600\n800490300\n001600050'),  "
     "    ('007200003\n000057860\n090300000\n709001200\n060070090\n001400306\n000006080\n046710000\n100003600'),  "
     "    ('000846532\n635297148\n284315967\n476528391\n398164725\n521739486\n163452879\n849671253\n752983614')   "
     "; "
)

In [ ]:
engine.insertData( add_user['1'] )

In [ ]:
engine.insertData( add_user['2'] )

In [ ]:
engine.insertData( add_sudokuboard['1'] )

## Búsqueda de datos

Dentro de la base de datos recién creada

In [4]:
query = "SELECT * FROM Binacle;"
engine.selectHeadRows(query)

[(1,
  'admin',
  'El usuario admin ha sido creado',
  datetime.datetime(2021, 4, 28, 10, 38, 38)),
 (2,
  'admin',
  'El usuario Renata ha sido creado',
  datetime.datetime(2021, 4, 28, 10, 39, 5)),
 (3,
  'admin',
  'El usuario Angie ha sido creado',
  datetime.datetime(2021, 4, 28, 10, 39, 5)),
 (4,
  'admin',
  'El usuario Clarissa ha sido creado',
  datetime.datetime(2021, 4, 28, 10, 39, 5)),
 (5,
  'admin',
  'El usuario Melissa ha sido creado',
  datetime.datetime(2021, 4, 28, 10, 39, 5))]

In [5]:
query = "SELECT * FROM User;"
engine.selectHeadRows(query)

[(1, 'admin', 1, 'D6434E01BBA64B1E49C840CE733FE794', 1),
 (2, 'Renata', 0, 'admin', 1),
 (3, 'Angie', 0, 'admin', 1),
 (4, 'Clarissa', 0, 'admin', 1),
 (5, 'Melissa', 0, 'admin', 1)]